In [ ]:
!pip install transformers datasets accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.

In [ ]:
import transformers
import datasets
from datasets import load_dataset
dataset = load_dataset("SetFit/sst5")

# dataset = dataset.rename_column("label", "label_num")

from datasets import concatenate_datasets
dataset_train = concatenate_datasets([ dataset["train"] , dataset["validation"]])
dataset_test = dataset["test"]
model_checkpoint = "distilbert/distilbert-base-uncased"

Repo card metadata block was not found. Setting CardData to empty.


Generating train split:   0%|          | 0/8544 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1101 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2210 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
import re

max_input_length = 512
max_target_length = 200


def clean_text(text):

  # to clean data, since its already pretty clean, not a lot of cleaning is necessary
  # text = re.sub(r'[^a-zA-Z\s]', '', text)
  text = re.sub(r'\s+', ' ', text).strip()

  return text

def preprocess_data(examples):
  texts_cleaned = [clean_text(text) for text in examples["text"]]
  model_inputs = tokenizer(texts_cleaned, max_length=max_input_length, truncation=True, padding = True)
  # targets = [str(label) for label in examples["labels"]]

  # # To setup the tokenizer for targets
  # with tokenizer.as_target_tokenizer():
  #       labels = tokenizer(targets, max_length=max_target_length, truncation=True, padding=False)

#   labels = tokenizer(text_target = examples["label_text"], max_length=max_target_length, truncation=True, padding = True)

#   model_inputs["labels"] = labels["input_ids"]
  return model_inputs

In [ ]:
tokenized_dataset = dataset_train.map(preprocess_data, batched=True)
tokenized_test_dataset = dataset_test.map(preprocess_data, batched=True)

Map:   0%|          | 0/9645 [00:00<?, ? examples/s]

Map:   0%|          | 0/2210 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset = tokenized_dataset.remove_columns(["label_text", "text"])
tokenized_test_dataset = tokenized_test_dataset.remove_columns(["label_text", "text"])

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, TrainingArguments,  AutoModelForSequenceClassification, DataCollatorWithPadding, Trainer


batch_size = 8
# model_name = "mt5-base"
# model_dir = f"google/{model_name}"

model =  AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=5)


args = TrainingArguments(
    output_dir="distilbert-ft-sst5", #save the results of the runs on disk
    evaluation_strategy="epoch", #evaluate the model at the end of each epoch
    logging_strategy="epoch", #log the training loss and metrics at the end of each epoch
    logging_steps=100, #log every 100 steps
    save_strategy="epoch", #save the model at the end of each epoch
    learning_rate=4e-5, #learning rate for the optimizer
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01, #weight decay for the optimizer
    save_total_limit=1,
    num_train_epochs=8,
#     fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss"
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
data_collator = DataCollatorWithPadding(tokenizer = tokenizer)

trainer = Trainer(
    model = model,
    args=args,
    train_dataset= tokenized_dataset,
    eval_dataset=tokenized_test_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,1.234000,1.118023
2,0.933400,1.209094
3,0.652400,1.301694


Epoch,Training Loss,Validation Loss
1,1.234000,1.118023
2,0.933400,1.209094
3,0.652400,1.301694
4,0.410000,1.718068


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
trainer.push_to_hub()